In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import utils
import time

In [2]:
# 对原始模型做了一点小改动，去掉了最后一层的高斯激活。除此之外，这个网络与最初的LeNet-5一致。
# 输入图片是28*28的单通道灰度图片（手写数字识别）
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), 
    nn.Sigmoid(),
    nn.Linear(120, 84), 
    nn.Sigmoid(),
    nn.Linear(84, 10)
)
# net = nn.Sequential(
#     nn.Conv2d(1, 10, kernel_size=3, padding=1), 
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(10, 40, kernel_size=3, padding=1), 
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(40, 256, kernel_size=1), 
#     nn.ReLU(),
#     nn.Flatten(),
#     nn.Linear(256 * 7 * 7, 128), 
#     nn.ReLU(),
#     nn.Linear(128, 10)
# )

In [3]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [4]:
net

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
)

In [5]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, nn.Module):
        net.eval()  # 设置为评估模式
        if not device:
            device = next(net.parameters()).device
    # 正确预测的数量，总预测的数量
    y_hat_true_count = 0
    y_hat_num_count = 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            
            y_hat_true_count += utils.accuracy(net(X), y)
            y_hat_num_count += y.numel()
    return y_hat_true_count / y_hat_num_count

In [6]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型(在第六章定义)"""
    time_0 = time.time()
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print(f'training on: [{device}], [{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}]')
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    # 时间用时列表
    time_list = list()
    # 每批次数据量
    train_num_per_epoch = 0
    # 初始化参数：训练集损失，训练集准确率，测试集准确率
    train_l = train_acc = test_acc = None
    # 训练损失之和，训练准确率之和，样本数
    train_loss = train_hat_true_count = train_num_count = 0
    
    num_batches = len(train_iter)
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，样本数
        train_loss = 0
        train_hat_true_count = 0
        train_num_count = 0
        # 训练模式
        net.train()
        for i, (X, y) in enumerate(train_iter):
            start_time = time.time()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                train_loss += l * X.shape[0]
                train_hat_true_count += utils.accuracy(y_hat, y)
                train_num_count += X.shape[0]

            time_list.append(time.time() - start_time)
            
            train_l = train_loss / train_num_count
            train_acc = train_hat_true_count / train_num_count
            
        train_num_per_epoch = train_num_count
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        print(f'epoch: {epoch+1}/{num_epochs}, loss {train_l:.3f}, train acc {train_acc:.3f}, test acc {test_acc:.3f}')
    
    print(f'*** {train_num_per_epoch * num_epochs / sum(time_list):.1f} examples/sec '
          f'on {str(device)} - [{utils.time_s2dhms(sum(time_list))}], '
          f'all: [{utils.time_s2dhms(time.time() - time_0)}] ***')

In [7]:
lr, num_epochs, batch_size = 1.0, 10, 512
# lr, num_epochs, batch_size = 0.01, 10, 64
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size=batch_size)
train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-05-30 22:12:31]
epoch: 1/10, loss 2.328, train acc 0.101, test acc 0.100
epoch: 2/10, loss 2.257, train acc 0.140, test acc 0.394
epoch: 3/10, loss 1.259, train acc 0.496, test acc 0.594
epoch: 4/10, loss 0.935, train acc 0.634, test acc 0.627
epoch: 5/10, loss 0.810, train acc 0.687, test acc 0.612
epoch: 6/10, loss 0.711, train acc 0.722, test acc 0.720
epoch: 7/10, loss 0.670, train acc 0.737, test acc 0.691
epoch: 8/10, loss 0.646, train acc 0.749, test acc 0.735
epoch: 9/10, loss 0.610, train acc 0.764, test acc 0.695
epoch: 10/10, loss 0.586, train acc 0.774, test acc 0.754
*** 43920.3 examples/sec on cuda:0 - [0:00:00:13], all: [0:00:00:50] ***
